In [245]:
if torch.backends.mps.is_available():
  mps_device = torch.device("mps")
  x = torch.ones(1, device=mps_device)
  print(x)
  print("Running on MPS Device")
else:
  print("MPS device not found")

tensor([1.], device='mps:0')
Running on MPS Device


In [246]:
max_iters = 100000
eval_iters = 200
eval_interval = 300
learning_rate = 1e-2

In [247]:
with open ('data/shakespeare.txt', 'r', encoding='utf-8') as f:
  text = f.read()


In [248]:
print("length of text file = ", len(text))

length of text file =  1115393


In [249]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [250]:
chars = sorted(list(set(text)))

In [251]:
vocab_size = len(chars)

In [252]:
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [253]:
# Character level language model
# Encode characters to numbers

stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}

encode = lambda s: [stoi[c] for c in s] # take a string output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

In [254]:
print(encode ("hii there"))

[46, 47, 47, 1, 58, 46, 43, 56, 43]


In [255]:
print(decode (encode ("hii there")))

hii there


In [256]:
import torch
data = torch.tensor(encode(text), dtype = torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115393]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [257]:
n = int(0.9* len(data))
train_data = data[:n]
val_data = data[n:]
print(len(train_data), len(val_data))

1003853 111540


In [258]:
block_size = 8
train_data[: block_size+1]


tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [259]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
  context = x[:t+1]
  target = y[t]
  print(f"When input is {context}, output is {target}")

When input is tensor([18]), output is 47
When input is tensor([18, 47]), output is 56
When input is tensor([18, 47, 56]), output is 57
When input is tensor([18, 47, 56, 57]), output is 58
When input is tensor([18, 47, 56, 57, 58]), output is 1
When input is tensor([18, 47, 56, 57, 58,  1]), output is 15
When input is tensor([18, 47, 56, 57, 58,  1, 15]), output is 47
When input is tensor([18, 47, 56, 57, 58,  1, 15, 47]), output is 58


In [260]:
print(x,y)

tensor([18, 47, 56, 57, 58,  1, 15, 47]) tensor([47, 56, 57, 58,  1, 15, 47, 58])


In [261]:
torch.randint(10,(5,))

tensor([0, 7, 5, 6, 4])

In [262]:
torch.manual_seed(1337)
batch_size = 4 
block_size = 8

def get_batches(split):
  # generate a small batch of data of inputs and targets y
  data = train_data if split == 'train' else val_data
  ix = torch.randint(len(data) - block_size, (batch_size,))
  x = torch.stack([ data[i  : i+block_size] for i in ix])
  y = torch.stack([ data[i+1: i+block_size+1] for i in ix])
  x,y = x.to(mps_device), y.to(mps_device)
  return x,y

xb,yb = get_batches('train')
print("inputs:")
print(xb.shape)
print(xb)
print("targets:")
print(yb.shape)
print(yb)

print("---")

for b in range (batch_size):
  for t in range(block_size):
    context = xb[b, : t+1]
    target = yb[b,t]
    print(f"when input is {context.tolist()}, the target is: {target}")

inputs:
torch.Size([4, 8])
tensor([[53, 59,  6,  1, 58, 56, 47, 40],
        [49, 43, 43, 54,  1, 47, 58,  1],
        [13, 52, 45, 43, 50, 53,  8,  0],
        [ 1, 39,  1, 46, 53, 59, 57, 43]], device='mps:0')
targets:
torch.Size([4, 8])
tensor([[59,  6,  1, 58, 56, 47, 40, 59],
        [43, 43, 54,  1, 47, 58,  1, 58],
        [52, 45, 43, 50, 53,  8,  0, 26],
        [39,  1, 46, 53, 59, 57, 43,  0]], device='mps:0')
---
when input is [53], the target is: 59
when input is [53, 59], the target is: 6
when input is [53, 59, 6], the target is: 1
when input is [53, 59, 6, 1], the target is: 58
when input is [53, 59, 6, 1, 58], the target is: 56
when input is [53, 59, 6, 1, 58, 56], the target is: 47
when input is [53, 59, 6, 1, 58, 56, 47], the target is: 40
when input is [53, 59, 6, 1, 58, 56, 47, 40], the target is: 59
when input is [49], the target is: 43
when input is [49, 43], the target is: 43
when input is [49, 43, 43], the target is: 54
when input is [49, 43, 43, 54], the target

In [263]:
@torch.no_grad()
def estimate_loss():
  out = {}
  m.eval()
  for split in ['train', 'val']:
    losses = torch.zeros(eval_iters)
    for k in range(eval_iters):
      X,Y = get_batches(split)
      logits, loss = model (X,Y)
      losses[k] = loss.item()
    out[split] = losses.mean()
  m.train()
  return out 

In [264]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
  
  def __init__(self, vocab_size):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

  def forward(self, idx, targets=None):
    logits = self.token_embedding_table(idx) # (batch, time, channel)
    
    if targets == None:
      loss = None
    else:
      B,T,C = logits.shape
      logits = logits.view(B*T, C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits, targets)
    
    return logits, loss
  
  def generate(self, idx, max_new_tokens):
    # idx is (B,T)  array of indices in the current context

    for _ in range(max_new_tokens):
      logits, loss = self(idx)
      logits = logits[:, -1, :]
      probs = F.softmax(logits, dim=-1)
      idx_next = torch.multinomial(probs, num_samples=1)
      idx = torch.cat((idx, idx_next), dim=1)
    return idx

model = BigramLanguageModel(vocab_size)
m = model.to(mps_device)
logits,loss = m(xb, yb)
print(logits.shape)
print(loss)
context = torch.zeros((1,1) , dtype=torch.long, device=mps_device) 
print(decode(m.generate( context, max_new_tokens=100)[0].tolist()))

torch.Size([32, 65])
tensor(4.8948, device='mps:0', grad_fn=<NllLossBackward0>)

UNasE3QKdYMjKfxcq-PyQbRF.
jxuUfZWievNL:C&v-jkcECOIiyeg zbZAcQ?yObr&MkzeAmyFXSPHd,j&?oneOAvrFotKuLTDx


In [265]:
optimizer = torch.optim.AdamW(m.parameters(), lr = 1e-3)

In [266]:
# batch_size = 32
# for steps in range(10000):


# print(loss.item())

for iter in range(max_iters):
  if iter % eval_interval == 0:
    losses = estimate_loss()
    print(f"step {iter}: train loss {losses['train']:.4f} val loss {losses['val']:.4f}")
    
  xb, yb = get_batches('train')
  logits, loss = m(xb, yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

step 0: train loss 4.7331 val loss 4.7272
step 300: train loss 4.5009 val loss 4.5134
step 600: train loss 4.3064 val loss 4.2910
step 900: train loss 4.1099 val loss 4.1279
step 1200: train loss 3.9290 val loss 3.9131
step 1500: train loss 3.7882 val loss 3.7971
step 1800: train loss 3.6375 val loss 3.6273
step 2100: train loss 3.4978 val loss 3.5211
step 2400: train loss 3.3808 val loss 3.4120
step 2700: train loss 3.2777 val loss 3.3131
step 3000: train loss 3.2137 val loss 3.2193
step 3300: train loss 3.1012 val loss 3.1650
step 3600: train loss 3.0703 val loss 3.0901
step 3900: train loss 2.9990 val loss 3.0108
step 4200: train loss 2.9242 val loss 2.9351
step 4500: train loss 2.9139 val loss 2.9165
step 4800: train loss 2.8583 val loss 2.8723
step 5100: train loss 2.8057 val loss 2.8369
step 5400: train loss 2.7822 val loss 2.7876
step 5700: train loss 2.7357 val loss 2.7726
step 6000: train loss 2.7508 val loss 2.7329
step 6300: train loss 2.6784 val loss 2.7318
step 6600: train

In [267]:
context = torch.zeros((1,1) , dtype=torch.long, device=mps_device)
print(decode(m.generate( idx = context , max_new_tokens=500)[0].tolist()))


HEayo in mpery way avend ouburser sickes bokecard dstceny

He tw el fe oupise he, lbustselownthous;
I m w
T:
TIONTouly me Ed frks, g he itheland's oe, oghithet f, badogienthofathatey foueay wad,
ureisold array n
ICoyockind m murs, in mamybalorthyongmyooe, d Vofetthindy st
Hefil brveseay alsteanerm to, oupomp rede d pre h, gavitfithrer'GENUpsts lathind my:
Berouerse IOLUEDid nghathicerire.
In IS:
Yok, pequt f keithunghaned t
The orerrofe find ans I andoovyonon-hu he nd youlliler pt icis ig y onee
